本文的研究目的是 <code>情感分析</code>, 通过一些已经定义好的词典来给句子向量化
## 1.1 读取情感词库 boson nlp

In [1]:
def getSentimentWords(url):
    with open(url,'r',encoding='utf8') as f:
        sentiWord = f.readlines()
    eachLine=[i.replace('\n','') for i in sentiWord]
    sentiDict={}
    for line in eachLine:
        twoCom=line.split(" ")
        if len(twoCom)==2:
            sentiDict[twoCom[0]]=float(twoCom[1])
    return sentiDict

BosonWords=getSentimentWords("NLP/Dictionary/BosonNLP/BosonNLP_sentiment_score.txt")
iniLen=len(BosonWords)
print("the length of boson dictionary: ",iniLen)
BosonWords

the length of boson dictionary:  114766


{'最尼玛': -6.70400012637,
 '扰民': -6.49756445867,
 'fuck...': -6.32963390433,
 'RNM': -6.21861284426,
 'wcnmlgb': -5.96710044003,
 '2.5:': -5.90459648251,
 'Fxxk': -5.87247473641,
 'MLP': -5.87247473641,
 '吃哑巴亏': -5.77120419579,
 'IAQI': -5.77107837123,
 'MLGBD': -5.69408191501,
 'NNND': -5.66228462641,
 'MLGB.': -5.60457743583,
 '成甘': -5.60457743583,
 '最桑': -5.60457743583,
 '真无语': -5.60457743583,
 'ＴＭ': -5.60457743583,
 '次奥次奥次奥': -5.59258287133,
 'cnmd': -5.54446545761,
 'MBD': -5.50280109843,
 'NNDX': -5.48173951768,
 '水蛭': -5.48173951768,
 '美素丽': -5.48173951768,
 '草尼': -5.48173951768,
 '凌迟': -5.46005372985,
 '尼玛尼玛尼玛': -5.42622557462,
 '冠周炎': -5.41616190446,
 '加塞儿': -5.41616190446,
 '日尼玛': -5.41616190446,
 '朝仲': -5.41616190446,
 'MD...': -5.34746092937,
 'shit....': -5.34746092937,
 '人清': -5.34746092937,
 '风疹块': -5.34746092937,
 'ＴＭＤ': -5.34746092937,
 '真桑心': -5.30440443402,
 'NND...': -5.27532412618,
 '延机': -5.27532412618,
 '毛明天': -5.27532412618,
 '逆流': -5.27532412618,
 '肺病': -5.234152

## 1.2 读取同义词库，因为boson情感词库的词语太少了，难免遇到情感词不够或者匹配不到的情况,需要拓展一下情感词

In [2]:
import numpy as np
import nltk
import json
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
def findSyn(w):
    return [e.lemma_names('cmn') for e in wn.synsets(w, lang='cmn')]

def iniSynonym():
    path = 'NLP/Dictionary/拓展同义词典/cow-not-full.txt'
    words = [l.split('\t')[-1].strip() for l in open(path).readlines()]
    syn_wn = {}
    for w in words:
        try:
            w_ = w.replace('+', '')
            added = []
            syn_lst = findSyn(w)
            for l in syn_lst:
                l_ = l.copy()
                l_.remove(w_)
                added.extend([i.replace('+', '') for i in l_])
            if added:
                syn_wn[w_] = added
        except:
            pass

    with open('NLP/Dictionary/拓展同义词典/wordnetSyn.json', 'w') as f:
        json.dump(syn_wn, f)

    for k,v in syn_wn.items():
        syn_wn[k]=list(np.unique(v))

    print("the length of Synonym dictionary: ",len(syn_wn))
    print("the increased number of words in Boson: ", len(syn_wn)-iniLen)
    return syn_wn


syn_wn=iniSynonym()
syn_wn

[nltk_data] Downloading package wordnet to /Users/czc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/czc/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/czc/anaconda3/envs/DataAnalysis/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=n at offset=15178842.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
/Users/czc/anaconda3/envs/DataAnalysis/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=n at offset=15171858.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
/Users/czc/anaconda3/envs/DataAnalysis/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=n at offset=14869977.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
/Users/czc/anaco

the length of Synonym dictionary:  32077
the increased number of words in Boson:  -82689


{'䁖': ['䁖䁖', '注视', '看', '看一下', '看一看', '看看'],
 '䁖䁖': ['䁖', '注视', '看', '看一下', '看一看', '看看'],
 '〇': ['无', '零'],
 '双手叉腰的': ['两手叉腰的'],
 '导水管': ['导管', '桥管', '水渠', '沟渠', '渡槽', '高架渠'],
 '急急忙忙': ['快速移动'],
 '战壕足': ['壕沟足', '足浸病'],
 '挪用者': ['侵占者;', '占用者', '擅用者', '盗用者'],
 '教会属地': ['教会附属地'],
 '智慧财产': ['智慧财产权', '知识产权'],
 '智慧财产权': ['智慧财产', '知识产权'],
 '炉石': ['砌炉的石块'],
 '柏拉图年': ['大年'],
 '大炮': ['加农炮', '坦克大炮', '火炮', '重炮'],
 '重炮': ['大炮', '长炮'],
 '船蛆': ['凿船虫', '蛀船虫'],
 '凿船虫': ['船蛆', '蛀船虫'],
 '双壳类动物': ['双壳贝', '瓣鳃', '瓣鳃动物', '瓣鳃纲软体动物'],
 '帽贝科': ['帽螺科', '笠螺科'],
 '无齿蚌属': ['河蚌属', '淡水珍珠贝属'],
 '章鱼科': ['八腕科'],
 '腹足类动物': ['单壳软体动物', '腹足动物', '腹足纲软体动物'],
 '蚝': ['牡蛎'],
 '非互易的': ['单向的', '非交互的', '非倒易的', '非相互的'],
 '非大选年': ['小年'],
 '玉螺': ['海月贝'],
 '长牡蛎': ['牡蛎'],
 '剃刀蛤': ['竹蛏'],
 '无板亚纲': ['双神经目', '无板类', '无板纲', '无板纲软体动物', '沟腹纲'],
 '圆基': ['膀胱螺', '膀胱螺属'],
 '放单飞': ['单飞'],
 '大扇贝': ['海扇贝', '海洋扇贝'],
 '100万': ['百万'],
 '100周年庆典': ['100周年庆祝',
  '100周年庆祝会',
  '100周年纪念',
  '一百周年庆典',
  '一百周年庆祝',
  '一百周年庆祝会',
  '一百周年纪念',
  '百周年纪念'],
 '100周年庆

In [3]:
findSyn('抛弃'),findSyn('驱逐')

([['丢弃', '抛弃', '拋弃', '放弃', '离弃', '遗弃'],
  ['丢弃', '丢掉', '弃置', '抛', '抛弃', '拋弃'],
  ['丢弃', '抛弃'],
  ['抛弃', '放弃'],
  ['抛弃', '放弃'],
  ['抛弃', '驱逐'],
  ['不坚持', '弃绝', '抛弃', '摈绝', '放弃']],
 [['放逐', '流亡', '流放', '驱逐', '驱逐出境'],
  ['赶出', '逐出', '驱逐'],
  ['驱散', '驱逐'],
  ['赶出', '赶走', '驱逐'],
  ['去掉', '排除', '摈弃', '消除', '清除', '逐出', '驱逐', '驱除'],
  ['废黜', '开除', '罢黜', '逐出', '驱逐'],
  ['抛弃', '驱逐'],
  ['开除', '逐出', '驱逐'],
  ['引渡', '移交', '驱逐', '驱逐出境'],
  ['驱逐'],
  ['排斥', '放逐', '驱逐'],
  ['放逐', '流放', '驱逐'],
  ['放逐', '驱逐']])

In [4]:
syn_wn['抛弃'],syn_wn['驱逐']

(['不坚持', '丢弃', '丢掉', '弃绝', '弃置', '抛', '拋弃', '摈绝', '放弃', '离弃', '遗弃', '驱逐'],
 ['去掉',
  '废黜',
  '开除',
  '引渡',
  '抛弃',
  '排斥',
  '排除',
  '摈弃',
  '放逐',
  '流亡',
  '流放',
  '消除',
  '清除',
  '移交',
  '罢黜',
  '赶出',
  '赶走',
  '逐出',
  '驱散',
  '驱逐出境',
  '驱除'])

In [5]:
###todo 整理同义词典，把同义词加到情感词典里面并赋值
for k in list(BosonWords.keys()):
    if k in syn_wn:
        synWordList=syn_wn[k]
        for word in synWordList:
            if word not in BosonWords:
                BosonWords[word]=BosonWords[k]

print("The final length of Boson dictionary: ",len(BosonWords))

The final length of Boson dictionary:  122471


# 1.3 加载程度副词

In [6]:
def separateListByElement(list_ori,p):
    list_new=[]				#处理后的列表，是一个二维列表
    list_short=[]			#用于存放每一段列表
    for i in list_ori:
        if i!=p:
            list_short.append(i)
        else:
            list_new.append(list_short)
            list_short=[]
    list_new.append(list_short)   #最后一段遇不到切割标识，需要手动放入
    return list_new

def getDegreeWords(url,fileTYpe="GBK"):
    with open(url,'r',encoding=fileTYpe) as f:
        sentiWord = f.readlines()
    allContent=separateListByElement(sentiWord,"-----\n")
    eachPart=[[j.replace('\n','') for j in i] for i in allContent]
    degreeDict={}
    for part in eachPart:
        for element in part:
            if "###" in element:
                degreeDict[element.replace("###","")]=[i for i in part if i!=element]
    return degreeDict

degreeWords=getDegreeWords("NLP/Dictionary/知网/程度级别词语（中文）.txt")
print(degreeWords.keys())
degreeScoreDict={"欠":0.6,'稍':0.9,'较':1.2,'很':1.6,'超':2.0,'极其/最':2.4}

dict_keys(['极其/最', '很', '较', '稍', '欠', '超'])


# 1.4 加载否定词

In [7]:
###stop word
def getStopWords(url):
    with open(url,'r',encoding='utf8') as f:
        stopword = f.readlines()
    return [i.replace('\n','') for i in stopword]

negativeWords=getStopWords("NLP/Dictionary/否定词典/not.txt")

# 1.5 加载主观词

In [8]:
subjectWords=getDegreeWords("NLP/Dictionary/知网/主张词语（中文）.txt")

# 1.6 给结巴分词加载停用词,情感词,程度副词，否定词

In [9]:
import jieba

cn_stopwords=getStopWords("NLP/stopwords/cn_stopwords.txt")
baidu_stopword=getStopWords("NLP/stopwords/baidu_stopwords.txt")
hit_stopword=getStopWords("NLP/stopwords/hit_stopwords.txt")
scu_stopword=getStopWords("NLP/stopwords/scu_stopwords.txt")

stopword_list=cn_stopwords+hit_stopword+baidu_stopword+scu_stopword
for stopword in stopword_list:
    jieba.add_word(stopword, freq=None, tag=None)
for k,v in BosonWords.items():
    jieba.add_word(k, freq=None, tag=None)
for k,v in degreeWords.items():
    for word in v:
        jieba.add_word(word, freq=None, tag=None)
for word in negativeWords:
    jieba.add_word(word)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ct/s3lrfn2541b8hydxbp_njzch0000gn/T/jieba.cache
Loading model cost 0.381 seconds.
Prefix dict has been built successfully.


# 1.7 测试句子
说明一下输出
>degree word -- 程度副词，例如 非常，很。
>sentiment word -- 情感词，例如 开心。
>subjectivity word -- 主张词，例如 感觉，表示主观。
>negative word -- 否定词，例如 不，非

In [10]:
import re

###这个数组的句子是测试用的，可以随意改该数组
testPara=['我今天很开心',
          '我今天非常开心，买到了自己想要的东西',
          '该商品很好用，很快。不过有些功能用的不是很明白，还需要改进一下。',
          '我今天不开心',
          '他觉得这个产品一点不好用',
          '他觉得这个产品很没用，最起码的功能一个都没有，这还怎么用',
          '尽管客户不喜欢这个产品，感觉使用起来有些困难。但是许多人反映该产品是第一次出现，还是震惊了许多厂商',
          '两个字难吃，三个字非常难吃，羊肉骚的离谱，牛肉不知道是什么鬼肉']

def calculateNegativeValue(myList) :
    if (len(myList) % 2) == 0:
        return 1
    else:
        return -1

def getSentiScoreOfSentence(wholeSentence:str)->float:
    theScores=[]###每个组合词的分数
    subSentences=[i for i in re.split('，|,|；|;',wholeSentence.strip()) if i.strip() != ""]

    for sentence in subSentences:
        ex="///".join(jieba.cut(sentence,cut_all=False))
        cutKeywords=ex.split('///')
        subScore=[]
        degreeWord=[]###提取出的程度副词
        handledWord=[]###已经处理了的词语
        sentiWords=[]###情感词
        subjeWords=[]###主观词
        negaWord=[]
        ###提取主观词
        for key in cutKeywords:
            isFound=False
            for k,v in subjectWords.items():
                if key in v and not isFound:
                    subjeWords.append(key)
                    isFound=True
        for key in cutKeywords:
            isFound=False
            if key in negativeWords and not isFound:
                negaWord.append(key)
                isFound=True
        ###todo 过滤程度副词，这里应该插入程度副词的识别，例如找到“非常”，就要看看后面的一个词是否是情感词，如果是，情感为“非常”程度向量*情感向量，提前加入theScores里面，然后从keyword里面去除这两个词语
        for index,item in enumerate(cutKeywords):
            isFound=False
            for key, value in degreeWords.items():
                ###假设程度副词+情感词
                if (item in value) and (cutKeywords[index+1] in BosonWords) and (index+1<=len(cutKeywords)-1):
                    if isFound==False:
                        subScore.append(degreeScoreDict[key]*BosonWords[cutKeywords[index+1]])
                        degreeWord.append(cutKeywords[index])
                        sentiWords.append(cutKeywords[index+1])

                        handledWord.append(cutKeywords[index])
                        handledWord.append(cutKeywords[index+1])

                        isFound=True
                ###这段单纯统计程度副词
                elif item in value:
                    if not isFound:
                        degreeWord.append(cutKeywords[index])
                        isFound=True
                ###假设程度副词+否定词+情感词


        afterHandledWords=[]###剩下未处理的词语
        for key in cutKeywords:
            if key not in handledWord:
                afterHandledWords.append(key)

        afterHandledWords=[j for j in afterHandledWords if j not in stopword_list]

        for keyword in afterHandledWords:
            if keyword in BosonWords:
                subScore.append(BosonWords[keyword])
                sentiWords.append(keyword)

        print("\n"
              "     child sentence: {}\n"
              "     keywords: {}\n"
              "     sentiment words: {}\n"
              "     degree word: {}\n"
              "     negative word: {}\n"
              "     subjective word: {}\n"
              "     sentiment score: {}\n"
              "     sum score: {}".format(sentence,
                            cutKeywords,
                            sentiWords,
                            degreeWord,
                            negaWord,
                            subjeWords,
                            subScore*calculateNegativeValue(negaWord),
              sum(subScore)*calculateNegativeValue(negaWord)))

        theScores.append(sum(subScore)*calculateNegativeValue(negaWord))

    return sum(theScores)

for para in testPara:
    print("content: ",para)
    finalScore=0
    ### 先切割成 句子级
    sentences=[i for i in re.split('!|。',para.strip()) if i.strip()!=""]

    for sentence in sentences:
        score=getSentiScoreOfSentence(sentence.strip())
        finalScore+=score

    print("\nscore: {}\nNorP: {}\n---------------".format(finalScore,"Positive" if finalScore>1 else "Negative"))

content:  我今天很开心

     child sentence: 我今天很开心
     keywords: ['我', '今天', '很', '开心']
     sentiment words: ['开心']
     degree word: ['很']
     negative word: []
     subjective word: []
     sentiment score: [4.179746770768]
     sum score: 4.179746770768

score: 4.179746770768
NorP: Positive
---------------
content:  我今天非常开心，买到了自己想要的东西

     child sentence: 我今天非常开心
     keywords: ['我', '今天', '非常', '开心']
     sentiment words: ['开心']
     degree word: ['非常']
     negative word: []
     subjective word: []
     sentiment score: [6.2696201561519995]
     sum score: 6.2696201561519995

     child sentence: 买到了自己想要的东西
     keywords: ['买到', '了', '自己', '想要', '的', '东西']
     sentiment words: ['想要', '东西']
     degree word: []
     negative word: []
     subjective word: []
     sentiment score: [-0.900672204041, 0.926729066929]
     sum score: 0.02605686288800002

score: 6.295677019039999
NorP: Positive
---------------
content:  该商品很好用，很快。不过有些功能用的不是很明白，还需要改进一下。

     child sentence: 该商品很好用
     

# 2.0 Snownlp
>这是一个官方的接口，只要把字符串放进去就能得到情感值，不过还没试过。
就个人而言，我不太推荐使用该接口，跟接口技术没关系，是使用了该接口会减少许多原创性（纯调包），毕竟情感分析也是该项目的一大亮点